Let's first install the libraries which GAN uses to do the generation and training.

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ninja

Now we'll get the latest repo of GAN 

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Let us see what GPU we have

In [ ]:
!nvidia-smi

Now for the dataset, first we would need to convert all the images into single dimensions. In this case 512 x 512 px

In [ ]:
from PIL import Image
import os

path = "D:\Dataset\SKU110K_fixed\SKU110K_fixed\images\\"
dest_path = "D:\Dataset\SKU110K_fixed\SKU110K_fixed\converted_images_512\\"
dirs = os.listdir(path)

def resize():
    for item in dirs:
        if item == '.jpg':
            continue
        if os.path.isfile(path+item):
            image = Image.open(path+item)
            file_path, extension = os.path.splitext(path+item)
            size = image.size

            new_image_height = 512
            new_image_width = 512

            try:
              if not os.path.isfile(dest_path + item):
                print(dest_path + item)
                image = image.resize((new_image_height, new_image_width), Image.ANTIALIAS)
                image.save(dest_path + item, 'JPEG', quality=90)
            except Exception as e:
              print("Corrupted File : ",e)
              


resize()

Once done, we would create the dataset from it

In [1]:
stylegan2_git_repo = 'D:\Git_Repositories\stylegan2-ada-pytorch\\'

In [2]:
!python D:/Git_Repositories/stylegan2-ada-pytorch/dataset_tool.py --source D:/Dataset/SKU110K_fixed/SKU110K_fixed/converted_images_512/ --dest D:/Dataset/SKU110K_fixed/SKU110K_fixed/dataset/


 24%|██▍       | 2804/11738 [00:38<01:57, 76.33it/s]

## Starting Training GAN on new Dataset

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/Experiments"
DATA = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/dataset/"
SNAP = 10

# Build the command and run it
cmd = f"python D:/Git_Repositories/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

## Resume Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/Experiments"
NETWORK = "network-snapshot-000040.pkl"
RESUME = os.path.join(EXPERIMENTS, "00002-dataset-auto1", NETWORK)
DATA = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/dataset/"
SNAP = 2

# Build the command and run it
cmd = f"python D:/Git_Repositories/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}